<img src='img/logo.png'>
<img src='img/title.png'>

# Preprocessing

# Table of Contents
* [Preprocessing](#Preprocessing)
	* [Setup](#Setup)
	* [Example workflow with `MinMaxScaler`](#Example-workflow-with-MinMaxScaler)
	* [Scaling training and test data the same way](#Scaling-training-and-test-data-the-same-way)
	* [The effect of preprocessing on supervised learning](#The-effect-of-preprocessing-on-supervised-learning)
		* [`preprocessing.scale`](#preprocessing.scale)
* [Summary](#Summary)


## Setup

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['image.interpolation'] = "none"
np.set_printoptions(precision=3)
plt.rcParams['image.cmap'] = "gray"

import src.mglearn as mglearn

http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

Preprocessing is often desirable for the following reasons:
 * Input data may have many dimensions and/or colinear dimensions, making it desirable to simplify the fitting problem by reducing the number of columns (features)
 * Input data may not have the statistical properties that are ideal for fitting

The following cell shows how several different preprocessing scalers affect input feature statstics.  An overview of the [sklearn.preprocessing api can be found here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing).

The block beginning with:
```

for ax, scaler in zip(other_axes, [StandardScaler(), RobustScaler(),
                                   MinMaxScaler(), Normalizer(norm='l2')]
```

is looping over 4 different scalers:
 * `StandardScaler`: [Remove mean and create unit variance](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
 * `RobustScaler`: [Scaling robust to outliers in input data](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler)
 * `MinMaxScaler`: [Scale to the min / max range of each feature](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)
 * `Normalizer(norm='l2')`: [Normalize to a unit norm](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, RobustScaler
from src.mglearn.plot_helpers import cm2



X, y = make_blobs(n_samples=50, centers=2, random_state=4, cluster_std=1)
X += 3

plt.figure(figsize=(15, 8))
main_ax = plt.subplot2grid((2, 4), (0, 0), rowspan=2, colspan=2)

main_ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm2, s=60)
maxx = np.abs(X[:, 0]).max()
maxy = np.abs(X[:, 1]).max()

main_ax.set_xlim(-maxx + 1, maxx + 1)
main_ax.set_ylim(-maxy + 1, maxy + 1)
main_ax.set_title("Original Data")
other_axes = [plt.subplot2grid((2, 4), (i, j)) for j in range(2, 4) for i in range(2)]

for ax, scaler in zip(other_axes, [StandardScaler(), RobustScaler(),
                                   MinMaxScaler(), Normalizer(norm='l2')]):
    X_ = scaler.fit_transform(X)
    ax.scatter(X_[:, 0], X_[:, 1], c=y, cmap=cm2, s=60)
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title(type(scaler).__name__)

other_axes.append(main_ax)

for ax in other_axes:
    ax.spines['left'].set_position('center')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('center')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

plt.suptitle("scaling_data")
plt.show();

## Example workflow with `MinMaxScaler`

This example uses a single training data / test data split, with a `MinMaxScaler` putting features in the `(0, 1)` range.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    random_state=1)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)    # first .fit finds stats needed for scaling

In [ ]:
# Stats of the data before / after MinMaxScaler
X_train_scaled = scaler.transform(X_train)
print("transformed shape: %s" % (X_train_scaled.shape,))
print("per-feature minimum before scaling:\n %s" % X_train.min(axis=0))
print("per-feature maximum before scaling:\n %s" % X_train.max(axis=0))
print("per-feature minimum after scaling:\n %s" % X_train_scaled.min(axis=0))
print("per-feature maximum after scaling:\n %s" % X_train_scaled.max(axis=0))

In [ ]:
# transform test data
X_test_scaled = scaler.transform(X_test)
# print test data properties after scaling
print("per-feature minimum after scaling: %s" % X_test_scaled.min(axis=0))
print("per-feature maximum after scaling: %s" % X_test_scaled.max(axis=0))

## Scaling training and test data the same way

Make sure to call the scaler's `.fit` method separately for training and test data, so the data are not scaled by a different data set's statistics.

In [ ]:
from sklearn.datasets import make_blobs
# make synthetic data
X, _ = make_blobs(n_samples=50, centers=5, random_state=4, cluster_std=2)
# split it into training and test set
X_train, X_test = train_test_split(X, random_state=5, test_size=.1)

# plot the training and test set
fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].scatter(X_train[:, 0], X_train[:, 1],
                c='b', label="training set", s=60)
axes[0].scatter(X_test[:, 0], X_test[:, 1], marker='^',
                c='r', label="test set", s=60)
axes[0].legend(loc='upper left')
axes[0].set_title("original data")

# scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# visualize the properly scaled data
axes[1].scatter(X_train_scaled[:, 0], X_train_scaled[:, 1],
                c='b', label="training set", s=60)
axes[1].scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], marker='^',
                c='r', label="test set", s=60)
axes[1].set_title("scaled data")

# rescale the test set separately, so that test set min is 0 and test set max is 1
# DO NOT DO THIS! For illustration purposes only
test_scaler = MinMaxScaler()
test_scaler.fit(X_test)
X_test_scaled_badly = test_scaler.transform(X_test)

# visualize wrongly scaled data
axes[2].scatter(X_train_scaled[:, 0], X_train_scaled[:, 1],
                c='b', label="training set", s=60)
axes[2].scatter(X_test_scaled_badly[:, 0], X_test_scaled_badly[:, 1], marker='^',
                c='r', label="test set", s=60)
axes[2].set_title("improperly scaled data");

## The effect of preprocessing on supervised learning

The cells below show that min / max scaling or standardization can improve the predictive value of a support vector classifier for the cancer data set.

In [ ]:
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    random_state=0)

svm = SVC(C=100)
svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

In [ ]:
# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# learning an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scoring on the scaled test set

svm.score(X_test_scaled, y_test)

In [ ]:
# preprocessing using zero mean and unit variance scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# learning an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scoring on the scaled test set
svm.score(X_test_scaled, y_test)

### `preprocessing.scale`

Scaling with function instead of a class.  The default behavior removes the mean and scales standard deviation to 1.

In [ ]:
from sklearn import preprocessing
import numpy as np
train = np.c_[np.random.normal(10, 2, 100),
              np.random.normal(size=100),
              np.random.normal(0, 5, 100)]

# with_mean/with_std True/False (default True)
scaled = preprocessing.scale(train)
print(np.allclose(0.0, scaled.mean(axis=0)), np.allclose(1.0, scaled.std(axis=0)))

# Summary

In this notebook, we reviewed the following topics in preparation for more advanced topics:

* [Preprocessing](#Preprocessing)
* [Example workflow with `MinMaxScaler`](#Example-workflow-with-MinMaxScaler)
* [Scaling training and test data the same way](#Scaling-training-and-test-data-the-same-way)
* [The effect of preprocessing on supervised learning](#The-effect-of-preprocessing-on-supervised-learning)
* [`preprocessing.scale`](#preprocessing.scale)


<a href='Scaling_and_Normalization_Exercises.ipynb' class='btn btn-primary btn-lg'>Exercises</a>

<img src='img/copyright.png'>